In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

# assign to first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#VRAM memory on-demand usage(prevent Keras takes all memory at the beginning)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
sess = tf.Session(config=config)

KTF.set_session(sess)


Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras import regularizers
from keras.callbacks import ModelCheckpoint

import numpy as np

In [3]:
num_classes = 10
num_predictions = 20
batch_size = 64

(x_main, y_main), (x_test, y_test) = cifar10.load_data()

print('x shape:', x_main.shape)
x_train=x_main[:40000]
y_train=y_main[:40000]
print('x_train shape:', x_train.shape)
x_val=x_main[40000:]
y_val=y_main[40000:]
print('x_val shape:', x_val.shape)




# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_val=x_val.astype('float32')
x_test = x_test.astype('float32')




x shape: (50000, 32, 32, 3)
x_train shape: (40000, 32, 32, 3)
x_val shape: (10000, 32, 32, 3)


In [4]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [5]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

mc = ModelCheckpoint('no_z_model_.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [6]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training


opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=150,\
                    verbose=1,validation_data=(x_val,y_val),callbacks=[LearningRateScheduler(lr_schedule),mc])

#save to disk
model_json = model.to_json()
with open('no_z_model.json', 'w') as json_file:
    json_file.write(model_json)



Epoch 1/150
625/625 [==============================] - 19s 30ms/step - loss: 2.0455 - acc: 0.3984 - val_loss: 1.6453 - val_acc: 0.5004

Epoch 00001: val_acc improved from -inf to 0.50040, saving model to no_z_model_.h5
Epoch 2/150
625/625 [==============================] - 16s 26ms/step - loss: 1.5198 - acc: 0.5472 - val_loss: 1.3235 - val_acc: 0.6140

Epoch 00002: val_acc improved from 0.50040 to 0.61400, saving model to no_z_model_.h5
Epoch 3/150
625/625 [==============================] - 16s 26ms/step - loss: 1.3502 - acc: 0.6094 - val_loss: 1.5874 - val_acc: 0.6153

Epoch 00003: val_acc improved from 0.61400 to 0.61530, saving model to no_z_model_.h5
Epoch 4/150
625/625 [==============================] - 16s 26ms/step - loss: 1.2530 - acc: 0.6472 - val_loss: 1.3976 - val_acc: 0.6424

Epoch 00004: val_acc improved from 0.61530 to 0.64240, saving model to no_z_model_.h5
Epoch 5/150
625/625 [==============================] - 16s 26ms/step - loss: 1.1842 - acc: 0.6730 - val_loss: 1.169

625/625 [==============================] - 19s 30ms/step - loss: 0.7422 - acc: 0.8092 - val_loss: 0.6827 - val_acc: 0.8407

Epoch 00042: val_acc improved from 0.82080 to 0.84070, saving model to no_z_model_.h5
Epoch 43/150
625/625 [==============================] - 19s 31ms/step - loss: 0.7322 - acc: 0.8113 - val_loss: 0.9232 - val_acc: 0.7818

Epoch 00043: val_acc did not improve from 0.84070
Epoch 44/150
625/625 [==============================] - 19s 31ms/step - loss: 0.7302 - acc: 0.8128 - val_loss: 0.7887 - val_acc: 0.8075

Epoch 00044: val_acc did not improve from 0.84070
Epoch 45/150
625/625 [==============================] - 19s 31ms/step - loss: 0.7150 - acc: 0.8157 - val_loss: 0.7091 - val_acc: 0.8304

Epoch 00045: val_acc did not improve from 0.84070
Epoch 46/150
625/625 [==============================] - 19s 31ms/step - loss: 0.7237 - acc: 0.8147 - val_loss: 0.7784 - val_acc: 0.8001

Epoch 00046: val_acc did not improve from 0.84070
Epoch 47/150
625/625 [====================

625/625 [==============================] - 19s 31ms/step - loss: 0.5183 - acc: 0.8670 - val_loss: 0.5183 - val_acc: 0.8703

Epoch 00084: val_acc did not improve from 0.87920
Epoch 85/150
625/625 [==============================] - 19s 31ms/step - loss: 0.5106 - acc: 0.8686 - val_loss: 0.5349 - val_acc: 0.8675

Epoch 00085: val_acc did not improve from 0.87920
Epoch 86/150
625/625 [==============================] - 19s 31ms/step - loss: 0.5085 - acc: 0.8700 - val_loss: 0.5697 - val_acc: 0.8554

Epoch 00086: val_acc did not improve from 0.87920
Epoch 87/150
625/625 [==============================] - 19s 31ms/step - loss: 0.5025 - acc: 0.8686 - val_loss: 0.5586 - val_acc: 0.8592

Epoch 00087: val_acc did not improve from 0.87920
Epoch 88/150
625/625 [==============================] - 19s 31ms/step - loss: 0.4978 - acc: 0.8704 - val_loss: 0.5271 - val_acc: 0.8650

Epoch 00088: val_acc did not improve from 0.87920
Epoch 89/150
625/625 [==============================] - 19s 31ms/step - loss: 

625/625 [==============================] - 19s 30ms/step - loss: 0.4524 - acc: 0.8805 - val_loss: 0.4812 - val_acc: 0.8779

Epoch 00127: val_acc did not improve from 0.88200
Epoch 128/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4590 - acc: 0.8790 - val_loss: 0.5107 - val_acc: 0.8679

Epoch 00128: val_acc did not improve from 0.88200
Epoch 129/150
625/625 [==============================] - 19s 30ms/step - loss: 0.4546 - acc: 0.8803 - val_loss: 0.4779 - val_acc: 0.8786

Epoch 00129: val_acc did not improve from 0.88200
Epoch 130/150
625/625 [==============================] - 19s 31ms/step - loss: 0.4536 - acc: 0.8800 - val_loss: 0.5016 - val_acc: 0.8735

Epoch 00130: val_acc did not improve from 0.88200
Epoch 131/150
625/625 [==============================] - 19s 31ms/step - loss: 0.4549 - acc: 0.8795 - val_loss: 0.4755 - val_acc: 0.8815

Epoch 00131: val_acc did not improve from 0.88200
Epoch 132/150
625/625 [==============================] - 19s 31ms/step - l

In [7]:
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


10000/10000 [==============================] - 1s 88us/step

Test result: 87.590 loss: 0.491


In [8]:
model.save('no_z_final.h5')


In [11]:
#del model
from keras.models import load_model
model= load_model('no_z_model_.h5')
scores = model.evaluate(x_test, y_test, batch_size=64, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 1s 102us/step

Test result: 88.030 loss: 0.486
